In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.cluster import KMeans
import pickle as pickle
import streamlit as st
import os
from datetime import datetime
import squarify as squarify
import base64
import csv
import re
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

In [15]:
df = pd.read_csv("data/segmentasicustomerwsh.csv")
df = pd.DataFrame(df)
pd.DataFrame(df)

,id_transaksi,id_customer,tanggal_transaksi,jumlah_transaksi,total_transaksi,item_transaksi
0,T001,WSH001,1/7/2023,7,61500,C(3); KK(2); SL(2)
1,T002,WSH002,1/7/2023,13,116500,BM(2); BT(1); C(2); R(2); S(1); SB(2); SL(3)
2,T003,WSH003,1/7/2023,21,183000,BB(3); BH(3); BM(3); BT(3); KL(3); KU(3); MA(3)
3,T004,WSH004,1/7/2023,5,42500,C(5)
4,T005,WSH002,1/7/2023,6,50000,KK(2); KM(2); KU(2)
...,...,...,...,...,...,...
585,T586,WSH002,8/11/2023,8,74500,BB(2); C(1); KU(2); O(1); S(1); T(1)
586,T587,WSH008,8/11/2023,3,22500,BT(2); KL(1)
587,T588,WSH018,8/11/2023,9,82000,DK(4); KM(1); MP(2); PP(1); T(1)
588,T589,WSH019,8/11/2023,8,66500,BB(1); BT(2); KK(2); O(1); SB(1); T(1)


In [16]:
df['id_customer'] = df['id_customer'].str.strip().str.lower()
pd.DataFrame(df)

,id_transaksi,id_customer,tanggal_transaksi,jumlah_transaksi,total_transaksi,item_transaksi
0,T001,wsh001,1/7/2023,7,61500,C(3); KK(2); SL(2)
1,T002,wsh002,1/7/2023,13,116500,BM(2); BT(1); C(2); R(2); S(1); SB(2); SL(3)
2,T003,wsh003,1/7/2023,21,183000,BB(3); BH(3); BM(3); BT(3); KL(3); KU(3); MA(3)
3,T004,wsh004,1/7/2023,5,42500,C(5)
4,T005,wsh002,1/7/2023,6,50000,KK(2); KM(2); KU(2)
...,...,...,...,...,...,...
585,T586,wsh002,8/11/2023,8,74500,BB(2); C(1); KU(2); O(1); S(1); T(1)
586,T587,wsh008,8/11/2023,3,22500,BT(2); KL(1)
587,T588,wsh018,8/11/2023,9,82000,DK(4); KM(1); MP(2); PP(1); T(1)
588,T589,wsh019,8/11/2023,8,66500,BB(1); BT(2); KK(2); O(1); SB(1); T(1)


In [17]:
print("Jumlah data yang memiliki missing null")
df.isnull().sum()
(df == 'NA').sum()

Jumlah data yang memiliki missing null


id_transaksi         0
id_customer          0
tanggal_transaksi    0
jumlah_transaksi     0
total_transaksi      0
item_transaksi       0
dtype: int64

In [18]:
print("Duplicate Data")
df.duplicated().sum()

Duplicate Data


0

In [19]:
df.replace('NA', pd.NA, inplace=True)
df.dropna(inplace=True)

In [20]:
#menghapus duplicate row
df.drop_duplicates(inplace=True)

In [21]:
print('Data Unik Tiap Row')
df.nunique()

Data Unik Tiap Row


id_transaksi         590
id_customer           38
tanggal_transaksi    131
jumlah_transaksi      18
total_transaksi      150
item_transaksi       348
dtype: int64

In [22]:
df['item_transaksi']=df['item_transaksi'].str.replace(';','')

In [23]:
df.reset_index(drop=True, inplace=True)
df['item_transaksi'] = df['item_transaksi'].apply(lambda x: [re.sub(r'\(\d+\)', ',', item).strip().rstrip(',') for item in x.split(',') ])

In [24]:
df

,id_transaksi,id_customer,tanggal_transaksi,jumlah_transaksi,total_transaksi,item_transaksi
0,T001,wsh001,1/7/2023,7,61500,"[C, KK, SL]"
1,T002,wsh002,1/7/2023,13,116500,"[BM, BT, C, R, S, SB, SL]"
2,T003,wsh003,1/7/2023,21,183000,"[BB, BH, BM, BT, KL, KU, MA]"
3,T004,wsh004,1/7/2023,5,42500,[C]
4,T005,wsh002,1/7/2023,6,50000,"[KK, KM, KU]"
...,...,...,...,...,...,...
585,T586,wsh002,8/11/2023,8,74500,"[BB, C, KU, O, S, T]"
586,T587,wsh008,8/11/2023,3,22500,"[BT, KL]"
587,T588,wsh018,8/11/2023,9,82000,"[DK, KM, MP, PP, T]"
588,T589,wsh019,8/11/2023,8,66500,"[BB, BT, KK, O, SB, T]"


In [25]:
data = df.copy()
data = data.explode('item_transaksi')
data['item_transaksi'] = data['item_transaksi'].apply(lambda x: x.strip('()').split(', ') if isinstance(x, str) else x)
item_list = data['item_transaksi'].tolist()
item_list

[['C', 'KK', 'SL'],
 ['BM', 'BT', 'C', 'R', 'S', 'SB', 'SL'],
 ['BB', 'BH', 'BM', 'BT', 'KL', 'KU', 'MA'],
 ['C'],
 ['KK', 'KM', 'KU'],
 ['P', 'PP', 'S'],
 ['MA', 'MP', 'O', 'PG'],
 ['BB', 'BH', 'BM', 'C'],
 ['SB', 'SL'],
 ['BB', 'BM', 'KL', 'KK'],
 ['KK', 'KM', 'S'],
 ['C', 'KM'],
 ['SB', 'SL'],
 ['BH', 'BM'],
 ['BB', 'BH', 'BM', 'P', 'SB'],
 ['KL', 'KK', 'SB'],
 ['C'],
 ['KK', 'KM', 'KU'],
 ['SL', 'SM'],
 ['BH', 'BM', 'P', 'SB'],
 ['KL'],
 ['KU', 'SM'],
 ['O'],
 ['BB', 'BH', 'BM', 'P'],
 ['MA', 'MP', 'O'],
 ['KM'],
 ['P', 'PP'],
 ['BH', 'P', 'SB'],
 ['MA', 'PG'],
 ['BH', 'KL', 'KK'],
 ['BH', 'BM'],
 ['SB', 'SL', 'SM'],
 ['BH', 'PG'],
 ['S', 'SB', 'SL', 'SM'],
 ['KK', 'P'],
 ['P', 'PP'],
 ['MA', 'MP', 'O', 'PG'],
 ['MA', 'O'],
 ['BH', 'BM', 'SD'],
 ['KM', 'MA', 'MP', 'O'],
 ['BH', 'KK', 'P', 'SB'],
 ['P', 'SB'],
 ['BH', 'KK'],
 ['KK'],
 ['BM', 'SM'],
 ['BH', 'KK'],
 ['BT', 'DK', 'KM', 'MA', 'PG', 'R'],
 ['KK'],
 ['KM', 'KU', 'MA', 'MP'],
 ['BB', 'BH', 'SM', 'SD'],
 ['KM', 'O', 'PG'],


In [26]:
dfitem = pd.DataFrame(item_list)
frequent_item = fpgrowth(dfitem, min_support=0.1, use_colnames=True)

C:\Users\handr\AppData\Roaming\Python\Python311\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value C

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(item_list).transform(item_list)
df = pd.DataFrame(te_ary, columns=te.columns_)
print("Item Per Transaksi Dalam Setiap Transaksi")
df

Item Per Transaksi Dalam Setiap Transaksi


,BB,BH,BM,BT,C,DK,F,KK,KL,KM,...,P,PG,PP,R,S,SB,SD,SL,SM,T
0,False,False,False,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
1,False,False,True,True,True,False,False,False,False,False,...,False,False,False,True,True,True,False,True,False,False
2,True,True,True,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,True,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
586,False,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
587,False,False,False,False,False,True,False,False,False,True,...,False,False,True,False,False,False,False,False,False,True
588,True,False,False,True,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,True


In [ ]:
rank_item = df.sum().sort_values(ascending=False).reset_index()
rank_item.columns = ['item', 'frequency']
rank_item

,item,frequency
0,BB,135
1,BH,133
2,KK,119
3,BM,112
4,KM,94
5,MA,92
6,MP,91
7,C,90
8,KL,88
9,KU,83


In [ ]:
frequent_item = fpgrowth(df, min_support=0.03, use_colnames=True)
frequent_item['itemsets'] = frequent_item['itemsets'].apply(lambda x: list(x))
frequent_item

,support,itemsets
0,0.201695,[KK]
1,0.152542,[C]
2,0.077966,[SL]
3,0.189831,[BM]
4,0.118644,[BT]
5,0.111864,[SB]
6,0.083051,[S]
7,0.081356,[R]
8,0.228814,[BB]
9,0.225424,[BH]


In [ ]:
rules = association_rules(frequent_item, metric="confidence", min_threshold=0.05)
rules = rules.drop_duplicates(subset=['antecedents', 'consequents']).sort_values(by='confidence', ascending=False).reset_index(drop=True)
rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x))
rules['consequents'] = rules['consequents'].apply(lambda x: list(x))
rules['rule_description'] = rules.apply(lambda x: f"Jika Membeli {x['antecedents']} maka membeli {x['consequents']}", axis=1)        
rules[['rule_description', 'support', 'confidence', 'lift']]

,rule_description,support,confidence,lift
0,"Jika Membeli ['BB', 'BM'] maka membeli ['BH']",0.040678,0.545455,2.419686
1,"Jika Membeli ['BB', 'BH'] maka membeli ['BM']",0.040678,0.480000,2.528571
2,"Jika Membeli ['BM', 'BH'] maka membeli ['BB']",0.040678,0.470588,2.056645
3,Jika Membeli ['BM'] maka membeli ['BH'],0.086441,0.455357,2.020005
4,Jika Membeli ['S'] maka membeli ['C'],0.033898,0.408163,2.675737
5,Jika Membeli ['BM'] maka membeli ['BB'],0.074576,0.392857,1.716931
6,Jika Membeli ['BH'] maka membeli ['BM'],0.086441,0.383459,2.020005
7,Jika Membeli ['BH'] maka membeli ['BB'],0.084746,0.375940,1.642996
8,Jika Membeli ['BB'] maka membeli ['BH'],0.084746,0.370370,1.642996
9,Jika Membeli ['T'] maka membeli ['BB'],0.038983,0.365079,1.595532
